In [1]:
import pandas as pd
from os import listdir, popen
from os.path import isfile, join, basename,splitext
import pinyin
%matplotlib inline

In [2]:
#遍历目录
dataPath = "../Competition/赛题1数据集"
files = [f for f in listdir(dataPath) if isfile(join(dataPath, f))]

In [3]:
#读数据
allData = dict()
for f in files:
    fullPath = join(dataPath, f)
    print(fullPath)
    allData[splitext(f)[0]] = pd.read_excel(fullPath)

../Competition/赛题1数据集/专利.xlsx
../Competition/赛题1数据集/融资信息.xlsx
../Competition/赛题1数据集/购地-市场交易-土地抵押.xlsx
../Competition/赛题1数据集/年报-对外投资信息.xlsx
../Competition/赛题1数据集/商标.xlsx
../Competition/赛题1数据集/购地-结果公告.xlsx
../Competition/赛题1数据集/债券信息.xlsx
../Competition/赛题1数据集/项目信息.xlsx
../Competition/赛题1数据集/产品.xlsx
../Competition/赛题1数据集/购地-市场交易-土地转让.xlsx
../Competition/赛题1数据集/年报-企业基本信息.xlsx
../Competition/赛题1数据集/上市信息财务信息-现金流量表.xlsx
../Competition/赛题1数据集/年报-企业资产状况信息.xlsx
../Competition/赛题1数据集/上市信息财务信息-成长能力指标.xlsx
../Competition/赛题1数据集/年报-的对外提供保证担保信息.xlsx
../Competition/赛题1数据集/纳税A级年份.xlsx
../Competition/赛题1数据集/招投标.xlsx
../Competition/赛题1数据集/年报-社保信息.xlsx
../Competition/赛题1数据集/上市信息财务信息运营能力指标.xlsx
../Competition/赛题1数据集/购地-房地产大地块出让情况.xlsx
../Competition/赛题1数据集/购地-房地产大企业购地情况.xlsx
../Competition/赛题1数据集/竞品.xlsx
../Competition/赛题1数据集/企业评分.xlsx
../Competition/赛题1数据集/上市信息财务信息资产负债表.xlsx
../Competition/赛题1数据集/软著著作权.xlsx
../Competition/赛题1数据集/年报-网站或网点信息.xlsx
../Competition/赛题1数据集/年报-股东（发起人）及出资信息.xlsx
../Competition/赛题1

In [4]:
#训练数据的标注
score = allData['企业评分']
score.to_excel("real.xlsx", header=False, index=False, engine='xlsxwriter', float_format='%d', sheet_name='Sheet')

In [5]:
#专利数量
zl = allData['专利']
zl_count = zl.iloc[:,[0,1]].groupby('企业编号').count()

In [65]:
#合并到score
score = score.merge(zl_count,how = 'left', on='企业编号')

In [68]:
#融资次数，合并到score
rz = allData['融资信息']
rz_count = rz.iloc[:,[0,2]].groupby('企业编号').count()
score = score.merge(rz_count,how = 'left', on='企业编号')
score.head()

,企业编号,企业总评分,专利类型,轮次
0,1001,75.374276,4.0,1.0
1,1002,79.830122,2.0,2.0
2,1003,78.318264,1713.0,1.0
3,1004,83.253376,7.0,1.0
4,1005,83.291493,14.0,1.0


In [73]:
#摊薄净资产收益率
ylnl = allData['上市信息财务信息盈利能力指标']
roe = ylnl.iloc[:,[0,4]]
roe.iloc[:,1] = pd.to_numeric(roe.iloc[:,1].str.rstrip('%'), errors='coerce')
roe_mean = roe.groupby('企业编号').mean()

In [103]:
#合并到score
score = score.merge(roe_mean,how = 'left', on='企业编号')
score.head()

/data/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [106]:
#简单线性回归
from sklearn import linear_model

In [112]:
y=score.iloc[:,1]
X=score.iloc[:,[4]]

In [114]:
lm = linear_model.LinearRegression()
model = lm.fit(X,y)


In [115]:
lm.score(X,y)

0.0002780733833822602

In [116]:
lm.coef_

array([0.00065702])

In [134]:
#预测结果
predictions = lm.predict(X)
predictDf = pd.DataFrame({'a': score.企业编号,'b':predictions })
predictDf.to_excel("predict.xlsx", header=False, index=False, engine='xlsxwriter', float_format='%d', sheet_name='Sheet')

In [146]:
#输出误差，越小越好
cmd = 'java -jar  ../Competition/mission1_test.jar  ./real.xlsx ./predict.xlsx'
with popen(cmd) as f:
    err = float(f.read().strip())
    print(err)

5.0990195135927845
